In [1]:
import pandas as pd
import numpy as np
from sklearn import cluster
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

In [2]:
df = pd.read_csv('../data/nutritional_values.csv')
recetas = pd.read_csv('../data/output.csv',sep='|')
ing = pd.read_csv('../data/ingredients_100.csv')
ing_raw = pd.read_csv('../data/ingredientes.csv',sep='|')

In [5]:
df.sort_values(['energia'], ascending=False)[:10]

,Recipe_id,alcohol,energia,grasa,proteina,agua,fibra,carbohidratos,ácido graso 22:6 n-3 (ácido docosahexaenóico),"ácidos grasos, monoinsaturados totales",...,calcio,"hierro, total",potasio,magnesio,sodio,fósforo,ioduro,"selenio, total",zinc (cinc),Total_Grams
2685,36774,0.0,39803.848902,3840.344167,1337.154723,7895.201500,7.187362,60.675002,6.560000,2189.865286,...,0.962511,0.112559,30.513849,1.572492,63.565070,13.770885,0.000025,0.001127,0.145412,13154.80556
4549,55961,0.0,15161.960000,1594.631100,30.012000,54.927000,0.762000,215.715300,0.000000,1223.482900,...,0.341150,0.005115,0.384930,0.066000,1.575019,0.581840,0.000117,0.000026,0.002671,1904.00000
6466,64838,0.0,14655.434123,1622.888305,10.839180,401.746438,15.762373,49.329644,0.000000,1153.690924,...,0.748416,0.046975,1.388171,0.181216,3.855605,0.268910,0.000076,0.000004,0.001510,2110.27218
5810,60328,0.0,14055.409773,519.549848,585.338634,1601.624388,59.969362,1727.654502,1.656599,210.509391,...,0.668808,0.050027,15.383846,1.067202,16.061721,9.271147,0.000084,0.000339,0.047091,4543.26994
2411,32470,0.0,13988.721379,1432.393651,66.690607,1450.890888,39.063049,243.585857,0.000000,1093.917358,...,0.621437,0.021845,3.736932,0.241024,3.470175,1.075246,0.000129,0.000188,0.007589,3251.71975
5653,59743,0.0,13700.773380,1468.607451,158.221607,850.802088,5.921049,4.611857,0.000000,1012.565698,...,0.474076,0.017454,2.612992,0.369801,6.729722,1.734600,0.000382,0.000247,0.026663,2500.93825
5819,60369,0.0,13651.157819,1462.485819,64.100515,1759.494905,38.570596,100.017441,0.000000,1007.865298,...,2.291627,0.006456,4.751588,0.298618,2.899848,1.618848,0.000085,0.000015,0.007575,3456.86250
6685,67993,0.0,12301.600064,1362.644291,35.634287,329.105779,2.608573,13.838781,0.000000,951.522384,...,0.179680,0.005876,0.755300,0.052336,0.922572,0.587671,0.000057,0.000035,0.003739,1744.28579
1616,26161,0.0,12132.817159,1218.057052,210.799267,713.195343,11.439979,111.201818,0.000000,814.923180,...,2.427367,0.020545,3.147074,0.324029,7.213070,1.819549,0.000305,0.000370,0.017696,2292.82793
270,8427,0.0,11580.599560,1008.493873,53.160226,224.334318,11.350688,590.581878,0.000000,783.958592,...,0.165062,0.005189,1.778687,0.152632,0.631892,1.109918,0.000029,0.000049,0.002117,1894.64065


In [47]:
df[df['energia']< 500]

,Recipe_id,alcohol,energia,grasa,proteina,agua,fibra,carbohidratos,ácido graso 22:6 n-3 (ácido docosahexaenóico),"ácidos grasos, monoinsaturados totales",...,calcio,"hierro, total",potasio,magnesio,sodio,fósforo,ioduro,"selenio, total",zinc (cinc),Total_Grams
0,1,3.575,35.090000,0.052000,0.188000,35.081000,0.032000,2.232000,0.000000,0.000000,...,0.002240,0.000037,0.035810,0.002320,0.004270,0.003190,0.000001,0.000000,0.000000,42.00000
1,5,0.000,103.252526,5.622834,1.402017,38.449477,3.532494,10.663666,0.000000,1.943786,...,0.035885,0.000524,0.178044,0.031209,0.008544,0.040281,0.000003,0.000001,0.000351,58.33445
2,8,0.000,44.937500,0.373438,2.306250,155.175000,4.218750,7.909375,0.000000,0.027500,...,0.122750,0.004634,0.260313,0.039056,0.009669,0.051344,0.000003,0.000001,0.000601,210.62500
10,52,0.000,379.692886,7.568778,12.028687,35.270943,5.199375,63.506967,0.003664,2.598754,...,0.110551,0.002661,0.176134,0.043404,0.677052,0.140707,0.000009,0.000035,0.000861,124.51111
11,53,0.000,166.307500,15.307950,2.907813,114.152500,1.131000,4.631500,0.000000,11.752725,...,0.111325,0.003519,0.434355,0.085862,0.277535,0.065208,0.000006,0.000001,0.000301,138.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7170,74128,0.000,29.050000,2.088000,0.454000,44.577000,0.310000,2.152000,0.002050,1.575800,...,0.006646,0.000203,0.062270,0.007101,0.390530,0.007800,0.000007,0.000001,0.000015,53.00000
7171,74129,0.000,489.746192,38.621576,17.987798,266.915144,5.487000,17.799078,0.032707,15.965871,...,0.544341,0.009487,1.154531,0.060126,0.636044,0.378062,0.000027,0.000006,0.001996,348.04762
7172,74131,0.000,403.380000,9.052000,18.362500,1526.379000,24.413000,60.936000,0.000000,2.421750,...,0.231018,0.006632,3.696400,0.217400,4.306700,0.415820,0.000096,0.000012,0.003306,1655.00000
7184,74195,0.000,307.300000,0.004000,1.759000,3.018000,0.012000,75.057000,0.000000,0.001000,...,0.452580,0.217507,1.656420,0.000570,0.226560,0.226870,0.000000,0.000000,0.000014,80.00000


In [48]:
ing[ing['Recipe_id']==74131]

,Recipe_id,Ingrediente,Cantidad,Unidad,Indice,Grams,Total_Grams,Num_comensales
46044,74131,calabacín,1.0,unidad,603,300.0,300.0,1.0
46045,74131,berenjena,1.0,unidad,596,300.0,300.0,1.0
46046,74131,cebolla,2.0,unidad,610,100.0,200.0,1.0
46047,74131,pimiento,1.0,unidad,650,75.0,75.0,1.0
46048,74131,pimiento rojo,1.0,trozo,651,100.0,100.0,1.0
46049,74131,tomate,4.0,unidad,667,150.0,600.0,1.0
46050,74131,tomate frito,4.0,cucharadas soperas,929,17.0,68.0,1.0
46051,74131,aceite de oliva,1.0,chorro,410,2.0,2.0,1.0
46052,74131,sal,1.0,cucharada,908,10.0,10.0,1.0


In [31]:
macros = df[['energia',
       'grasa', 'proteina',
       'fibra', 'carbohidratos']].copy()
macros.head()

,energia,grasa,proteina,fibra,carbohidratos
0,35.090000,0.052000,0.188000,0.032000,2.232000
1,103.252526,5.622834,1.402017,3.532494,10.663666
2,44.937500,0.373438,2.306250,4.218750,7.909375
3,601.000000,3.170000,30.550000,8.750000,110.300000
4,1658.207052,153.715161,12.688162,5.849558,59.213363


In [32]:
clusters = 4

pipe = Pipeline([('scaler', StandardScaler()),('cluster',cluster.KMeans(n_clusters=clusters))])
macros['cluster'] = pipe.fit_predict(macros)

In [33]:
def print_cluster_stats(df, clusters):
    print('Items per cluster: ')
    items_cluster = [len(df[df['cluster']==cluster]) for cluster in range(clusters)]
    print(items_cluster)

In [34]:
print_cluster_stats(macros, clusters)

Items per cluster: 
[790, 1650, 86, 4660]


In [35]:
macros[macros['cluster']==2]

,energia,grasa,proteina,fibra,carbohidratos,cluster
270,11580.599560,1008.493873,53.160226,11.350688,590.581878,2
438,7122.800000,733.481700,22.769375,11.252000,126.782000,2
441,6933.320000,645.920000,54.400000,7.940000,257.310000,2
743,10087.500000,1037.075000,62.625000,17.000000,152.000000,2
784,10900.500000,1127.411500,195.605000,14.400000,22.800000,2
...,...,...,...,...,...,...
6956,9702.610064,1046.671291,44.684287,4.306573,55.445761,2
6957,6559.400064,670.174291,19.764287,15.908573,129.102861,2
7059,9802.250000,999.000000,25.000000,8.750000,203.700000,2
7082,10755.455000,627.190400,402.672125,30.898000,867.753000,2


In [38]:
macros.groupby('cluster')['energia'].describe()

,count,mean,std,min,25%,50%,75%,max
cluster,,,,,,,,
0,790.0,3108.249439,1428.674270,509.230000,2142.876470,2762.375000,3731.665000,11026.500000
1,1650.0,2614.754519,904.051238,960.000000,1970.517429,2434.540428,3046.947500,6496.500000
2,86.0,9929.639035,3836.941384,6123.360000,8111.296455,9705.055032,10718.779850,39803.848902
3,4660.0,927.524210,535.167713,-815.480187,506.150430,890.275883,1287.678584,2976.660000


In [39]:
macros.groupby('cluster')['grasa'].describe()

,count,mean,std,min,25%,50%,75%,max
cluster,,,,,,,,
0,790.0,116.018698,100.368925,0.0000,40.844128,96.908450,157.516507,533.445282
1,1650.0,172.151558,93.954782,6.0486,108.721677,154.258252,220.887099,603.487285
2,86.0,945.686235,412.632023,329.8000,681.031165,1006.833500,1048.674236,3840.344167
3,4660.0,48.727334,45.431405,0.0000,10.438604,38.309100,72.862932,300.620900


In [42]:
macros.groupby('cluster')['carbohidratos'].describe()

,count,mean,std,min,25%,50%,75%,max
cluster,,,,,,,,
0,790.0,405.140040,217.385690,19.394318,274.448222,357.500000,468.953797,1815.057352
1,1650.0,104.836458,82.804517,-25.758505,37.436439,86.893187,151.988238,404.545333
2,86.0,192.029779,281.621876,0.049521,27.858809,86.859434,228.952175,1727.654502
3,4660.0,77.742770,72.874503,-310.632501,19.450566,54.588611,117.637633,417.200000
